In [107]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
from  sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from  sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from  sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC,SVC
from  sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier


import tensorflow as tf

In [46]:
HRdata=pd.read_csv('D:/FilpRobo/WA_Fn-UseC_-HR-Employee-Attrition.csv')

In [47]:
HRdata

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,884,Research & Development,23,2,Medical,1,2061,...,3,80,1,17,3,3,5,2,0,3
1466,39,No,Travel_Rarely,613,Research & Development,6,1,Medical,1,2062,...,1,80,1,9,5,3,7,7,1,7
1467,27,No,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,2064,...,2,80,1,6,0,3,6,2,0,3
1468,49,No,Travel_Frequently,1023,Sales,2,3,Medical,1,2065,...,4,80,0,17,3,2,9,6,0,8


In [48]:
HRdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

In [56]:
HRdata.isna().sum().sum()

0

In [87]:
def onehot_encode(df,column):
    df=df.copy()
    dummies=pd.get_dummies(df[column],prefix=column)
    df=pd.concat([df,dummies],axis=1)
    df=df.drop(column,axis=1)
    return df 

In [96]:
def preprocess_inputs(df):
    df=df.copy()
    df=df.drop(['EmployeeCount','EmployeeNumber','Over18','StandardHours'],axis=1)
    
    df['Gender']=df['Gender'].replace({'Female':0,'Male':1})
    df['OverTime']=df['OverTime'].replace({'No':0,'Yes':1})
    df['Attrition']=df['Attrition'].replace({'No':0,'Yes':1})
    
    df['BusinessTravel']=df['BusinessTravel'].replace({'Non-Travel':0,'Travel_Rarely':1,'Travel_Frequently':2})
    
    for column in ['Department','EducationField','JobRole','MaritalStatus']:
        df=onehot_encode(df,column=column)
    
    y=df['Attrition']
    X=df.drop('Attrition',axis=1)
    
    X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.7,shuffle=True,random_state=1)
    
    scaler=StandardScaler()
    scaler.fit(X_train)
    X_train=pd.DataFrame(scaler.transform(X_train),index=X_train.index,columns=X_train.columns)
    X_test=pd.DataFrame(scaler.transform(X_test),index=X_test.index,columns=X_test.columns)
    
    return X_train,X_test,y_train,y_test

In [97]:
X_train,X_test,y_train,y_test=preprocess_inputs(HRdata)

In [98]:
X_train

,Age,BusinessTravel,DailyRate,DistanceFromHome,Education,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,...,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single
99,0.762337,-2.005534,-0.816540,1.742140,0.092884,-0.669480,0.843888,0.055811,0.384204,-0.063108,...,2.128910,-0.255377,-0.335309,-0.255377,-0.495745,-0.533521,-0.237630,-0.530514,1.088445,-0.690165
785,0.326292,-0.141311,1.652087,1.366162,1.078220,-1.586784,0.843888,-0.239963,0.384204,0.838814,...,-0.469724,-0.255377,-0.335309,-0.255377,-0.495745,-0.533521,-0.237630,-0.530514,1.088445,-0.690165
918,1.525416,1.722911,-1.436773,-0.012423,0.092884,1.165128,0.843888,0.844541,0.384204,2.642657,...,-0.469724,3.915780,-0.335309,-0.255377,-0.495745,-0.533521,-0.237630,1.884966,-0.918742,-0.690165
1335,0.217281,-0.141311,0.035051,1.240836,1.078220,1.165128,0.843888,-1.225876,0.384204,-0.063108,...,-0.469724,-0.255377,-0.335309,-0.255377,2.017168,-0.533521,-0.237630,1.884966,-0.918742,-0.690165
1182,-0.109753,-2.005534,0.180264,-1.015031,1.078220,1.165128,-1.184992,-1.620241,-0.998124,-0.063108,...,-0.469724,-0.255377,2.982320,-0.255377,-0.495745,-0.533521,-0.237630,-0.530514,1.088445,-0.690165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,-0.436787,1.722911,0.015361,-1.015031,1.078220,0.247824,-1.184992,0.893837,1.766532,-0.063108,...,-0.469724,-0.255377,-0.335309,-0.255377,-0.495745,-0.533521,-0.237630,-0.530514,1.088445,-0.690165
905,-0.872832,-0.141311,-0.311985,-1.015031,0.092884,1.165128,-1.184992,1.041724,-0.998124,1.740736,...,-0.469724,-0.255377,-0.335309,3.915780,-0.495745,-0.533521,-0.237630,1.884966,-0.918742,-0.690165
1096,0.326292,-0.141311,0.190109,-0.388401,-0.892452,0.247824,0.843888,-1.373763,0.384204,1.740736,...,-0.469724,3.915780,-0.335309,-0.255377,-0.495745,-0.533521,-0.237630,-0.530514,-0.918742,1.448930
235,0.653326,-0.141311,0.524838,0.864859,0.092884,1.165128,-1.184992,0.696654,0.384204,1.740736,...,-0.469724,3.915780,-0.335309,-0.255377,-0.495745,-0.533521,-0.237630,-0.530514,1.088445,-0.690165


In [23]:
X['Gender'].unique()

array(['Female', 'Male'], dtype=object)

In [77]:
X.select_dtypes('object')

""
0
1
2
3
4
...
1465
1466
1467
1468


In [78]:
{column: len(X[column].unique()) for column in X.select_dtypes('object').columns}

{}

In [21]:
pd.get_dummies(X['Gender'],prefix='Gender')

,Gender_Female,Gender_Male
0,1,0
1,0,1
2,0,1
3,1,0
4,0,1
...,...,...
1465,0,1
1466,0,1
1467,0,1
1468,0,1


In [94]:
X_train.mean()

Age                                  1.529928e-16
BusinessTravel                       3.538903e-17
DailyRate                           -3.261752e-17
DistanceFromHome                    -1.693926e-17
Education                            2.010054e-16
EnvironmentSatisfaction             -2.045659e-16
Gender                              -3.845321e-16
HourlyRate                           2.389839e-16
JobInvolvement                       1.592506e-16
JobLevel                            -2.801991e-16
JobSatisfaction                     -2.114711e-16
MonthlyIncome                       -4.898360e-17
MonthlyRate                         -4.553101e-17
NumCompaniesWorked                   8.739365e-17
OverTime                             9.181728e-17
PercentSalaryHike                   -3.700743e-17
PerformanceRating                    8.092005e-16
RelationshipSatisfaction             2.012212e-16
StockOptionLevel                     1.454403e-16
TotalWorkingYears                    1.085408e-16


In [99]:
y_train

99      0
785     0
918     0
1335    0
1182    0
       ..
715     0
905     0
1096    0
235     0
1061    0
Name: Attrition, Length: 1029, dtype: int64

In [109]:
models={
    "  Logistic Regression":LogisticRegression(),
    "  K-Nearest Neighbour":KNeighborsClassifier(),
    "       Decsision Tree":DecisionTreeClassifier(),
    "SupportVectorMachin(LK)":LinearSVC(),
    "SupportVectorMachin(RBF)":SVC(),
    "       Neural Network":MLPClassifier(),
    "        Random Forest":RandomForestClassifier(),
    "    Gradient Boosting":GradientBoostingClassifier()
}

for name,model in models.items():
    model.fit(X_train,y_train)
    print(name+" train")

  Logistic Regression train
  K-Nearest Neighbour train
       Decsision Tree train
SupportVectorMachin(LK) train
SupportVectorMachin(RBF) train
       Neural Network train
        Random Forest train
    Gradient Boosting train


In [111]:
for name,model in models.items():
    print(name+":{:2f}%".format(model.score(X_test,y_test)*100))

  Logistic Regression:86.621315%
  K-Nearest Neighbour:82.086168%
       Decsision Tree:78.231293%
SupportVectorMachin(LK):85.714286%
SupportVectorMachin(RBF):84.807256%
       Neural Network:84.580499%
        Random Forest:84.580499%
    Gradient Boosting:84.126984%
